# PlusMinus VQC — Blind Quantum Computing

Este notebook demonstra como treinar um modelo de Rede Neural Quântica (VQC)
usando o Qiskit Machine Learning e circuitos gerados a partir do dataset PlusMinus.

In [7]:
import sys
print(sys.executable)
!{sys.executable} -m pip install pennylane


c:\Users\Dreysv\Documents\GitHub\Hybrid_QML_with_BQC\.venv\Scripts\python.exe


In [8]:
#Instale as dependências 
!pip install qiskit qiskit-aer qiskit-machine-learning scikit-learn matplotlib numpy
!pip install qiskit-machine-learning
!pip install qiskit-algorithms
!pip install pennylane-datasets pennylane

ERROR: Could not find a version that satisfies the requirement pennylane-datasets (from versions: none)
ERROR: No matching distribution found for pennylane-datasets


In [9]:

import random
import numpy as np
import pennylane as qml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report

from qiskit import QuantumCircuit, transpile
from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes
from qiskit.primitives import Sampler
from qiskit_machine_learning.neural_networks import SamplerQNN
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier
#from qiskit_machine_learning.utils.loss_functions import CrossEntropyLoss
from qiskit_algorithms.optimizers import COBYLA

from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, depolarizing_error


# Controle de Aleatoriedade e Reprodutibilidade

In [10]:

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
try:
    from qiskit_algorithms.utils import algorithm_globals
    algorithm_globals.random_seed = SEED
except ImportError:
    pass

# Backend

In [11]:

noise_model = NoiseModel()
error_1q = depolarizing_error(0.01, 1)
error_2q = depolarizing_error(0.02, 2)
noise_model.add_all_qubit_quantum_error(error_1q, ['h','x','y','z','rx','ry','rz','s','sdg','t','tdg'])
noise_model.add_all_qubit_quantum_error(error_2q, ['cx','cz','swap'])

simulator = AerSimulator(noise_model=noise_model, shots=128, method='matrix_product_state')


# Função da Rede

In [12]:
def enviar_circuito_por_epoca(circuito, epoch, num_qubits, circuit_depth):
    print(f"[Epoch {epoch+1}] Enviando circuito para a rede...")
    try:
        rede.application_layer.run_app(
            "BFK_BQC",
            alice_id=6,
            bob_id=0,
            num_qubits=num_qubits,
            scenario=2,
            circuit_depth=circuit_depth,
            circuit=circuito
        )
        print(f"[Epoch {epoch+1}] Envio concluído.")
    except Exception as e:
        print(f"[Epoch {epoch+1}] Erro ao enviar circuito: {str(e)}")

# Carregar e preparar o dataset Plus-Minus

In [14]:
!pip install aiohttp fsspec h5py
[ds] = qml.data.load("plus-minus")

X = ds.img_train
y = ds.labels_train
X_test = ds.img_test
y_test = ds.labels_test

from sklearn.decomposition import PCA
# Flatten das imagens: (n_amostras, altura*largura)
X = X.reshape((X.shape[0], -1))
X_test = X_test.reshape((X_test.shape[0], -1))

# PCA reduzindo para 4 dimensões (4 qubits)
pca = PCA(n_components=4, random_state=SEED)
X = pca.fit_transform(X)
X_test = pca.transform(X_test)
#

X = X.reshape((X.shape[0], -1))
X_test = X_test.reshape((X_test.shape[0], -1))
scaler = MinMaxScaler((0,1))
X = scaler.fit_transform(X)
X_test = scaler.transform(X_test)

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=SEED, stratify=y
)

   ---------------------------------------- 0.0/2.9 MB ? eta -:--:--
   ---------------------------------------- 2.9/2.9 MB 15.2 MB/s  0:00:00

   -------- -------------------------------  2/10 [h5py]
   -------- -------------------------------  2/10 [h5py]
   -------- -------------------------------  2/10 [h5py]
   -------- -------------------------------  2/10 [h5py]
   -------- -------------------------------  2/10 [h5py]
   -------- -------------------------------  2/10 [h5py]
   ------------ ---------------------------  3/10 [fsspec]
   ------------ ---------------------------  3/10 [fsspec]
   ------------ ---------------------------  3/10 [fsspec]
   ------------ ---------------------------  3/10 [fsspec]
   ------------ ---------------------------  3/10 [fsspec]
   -------------------- -------------------  5/10 [attrs]
   -------------------- -------------------  5/10 [attrs]
   ---------------------------- -----------  7/10 [yarl]
   ------------------------------------ ---  9

# VQC

In [15]:
num_classes = len(np.unique(y))
num_qubits = X.shape[1]

feature_map = ZZFeatureMap(num_qubits, reps=1)
ansatz = RealAmplitudes(num_qubits, reps=1)
qc = QuantumCircuit(num_qubits)
qc.compose(feature_map, inplace=True)
qc.compose(ansatz, inplace=True)

def interpret(x):
    return x % num_classes

sampler = Sampler(options={"backend": simulator})

qnn = SamplerQNN(
    circuit=qc,
    sampler=sampler,
    input_params=feature_map.parameters,
    weight_params=ansatz.parameters,
    interpret=interpret,
    output_shape=num_classes,
)


vqc = NeuralNetworkClassifier(
    neural_network=qnn,
    optimizer=COBYLA(maxiter=1),  # rodar uma iteração por "epoch"
    loss=CrossEntropyLoss(),
    one_hot=True,
    callback=lambda weights, loss, step: enviar_circuito_por_epoca(
        qc.assign_parameters(weights), step
    ),
)

C:\Users\Dreysv\AppData\Local\Temp\ipykernel_9340\691303636.py:13: DeprecationWarning: The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.
  sampler = Sampler(options={"backend": simulator})
C:\Users\Dreysv\AppData\Local\Temp\ipykernel_9340\691303636.py:15: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  qnn = SamplerQNN(


# Treianmento e Métricas

In [16]:

EPOCHS = 3
print("\nTREINANDO VQC...")

for epoch in range(EPOCHS):
    vqc.fit(X_train, y_train)

    y_pred_val = vqc.predict(X_val)
    acc_val = accuracy_score(y_val, y_pred_val)
    print(f"[Epoch {epoch+1}] Accuracy validação: {acc_val:.4f}")

y_pred_test = vqc.predict(X_test)
print("\nRESULTADOS FINAIS NO TESTE:")
print("Acurácia:", accuracy_score(y_test, y_pred_test))
print(classification_report(y_test, y_pred_test))



TREINANDO VQC...
[Epoch 1] Accuracy validação: 0.3550
[Epoch 2] Accuracy validação: 0.3550
[Epoch 3] Accuracy validação: 0.3550

RESULTADOS FINAIS NO TESTE:
Acurácia: 0.395
              precision    recall  f1-score   support

           0       0.35      0.66      0.46        50
           1       0.29      0.08      0.12        50
           2       0.45      0.76      0.56        50
           3       0.50      0.08      0.14        50

    accuracy                           0.40       200
   macro avg       0.40      0.40      0.32       200
weighted avg       0.40      0.40      0.32       200

